letting an agent reason step-by-step and use multiple tools in a sequence to solve complex problems.

In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent,load_tools
from langchain_core.tools import Tool
from langchain.agents.agent_types import AgentType

In [2]:

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
SERP_API_KEY = os.getenv("SERP_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=GOOGLE_API_KEY)

In [4]:
def weather_tools(temp_str : str) -> str:
    temp = float(temp_str)
    if temp > 30:
        return "It's hot! Wear light clothes and stay hydrated."
    elif temp < 10:
        return "It's cold! Wear warm clothes."
    else:
        return "The weather is pleasant today"

In [8]:
custom_tools = [
    Tool(
        name="WeatherAdviceTool",
        func=weather_tools,
        description="Gives clothing suggestions based on temperature in Celsius."
    ),
    load_tools(
        ["serpapi"],
        llm=llm,
        serpapi_api_key=SERP_API_KEY
    )
]

In [9]:
# Flatten the custom_tools list if it contains nested lists
flat_tools = []
for t in custom_tools:
    if isinstance(t, list):
        flat_tools.extend(t)
    else:
        flat_tools.append(t)

agents = initialize_agent(
    llm=llm,
    tools=flat_tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [10]:
query = "What's the temperature in New York right now and what should I wear?"
response = agents.run(query)
print(response)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_19136\1368837131.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agents.run(query)




> Entering new AgentExecutor chain...
I need to find the temperature in New York and then use that information to provide clothing suggestions.
Action: Search
Action Input: "temperature in New York right now"I need to find the temperature in New York and then use that information to provide clothing suggestions.
Action: Search
Action Input: "temperature in New York right now"
Observation: {'type': 'weather_result', 'temperature': '58', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '60%', 'wind': '11 mph', 'location': 'New York, NY', 'date': 'Monday 3:00 AM', 'weather': 'Clear with periodic clouds'}
Thought:
Observation: {'type': 'weather_result', 'temperature': '58', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '60%', 'wind': '11 mph', 'location': 'New York, NY', 'date': 'Monday 3:00 AM', 'weather': 'Clear with periodic clouds'}
Thought:I now have the temperature in Fahrenheit. I need to convert it to Celsius to use the WeatherAdviceTool.
Action: Search
Action 